###### 古典MD (GAFF2/AM1BCC力場）を用いて密度の計算を行う 

In [1]:
# !rm -r -f input.acpype
# !ls ./ |  grep -v -E 'inputs|.ipynb|MD_trajectories' | xargs rm -rf 
# !mkdir output

In [1]:
import pandas as pd
poly = pd.read_csv("../../smiles/PG6.csv")
#poly = poly[1:].reset_index(drop=True)

In [2]:
poly

,Smiles,Name,density
0,OCC(C)OCC(C)OCC(C)OCC(C)OCC(C)OCC(C)O[H],PG6,1.006


In [12]:
max_atoms=63*600
density = 0.01

dt = 1                           #[fs] MDの刻み時間：このまま使うことを推奨。
eq_temp = 300.0                    #緩和計算させるときの温度 [K]
eq_steps = 10100000             #緩和計算するstep数。この例だと1.1nsec.
eq_cutoff = 5.0

!. /opt/gromacs/bin/GMXRC
gromacs_home = "/opt/gromacs/bin/"
gromacs_home = "/opt/homebrew/bin/"

zsh:.:1: no such file or directory: /opt/gromacs/bin/GMXRC


In [4]:
import pandas as pd

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools

#化学構造のsmilesリストを準備する
smiles_list_polymer = poly["Smiles"].to_list()

# 化合物のラベルを作成
label_list = poly["Name"].to_list()

# molオブジェクトのリストを作成
mols_list_polymer = [Chem.MolFromSmiles(smile) for smile in smiles_list_polymer]

In [4]:
def gaff_bcc(smiles):
    #GAFF2/AM1-BCCをアサインする
    import sys,os,os.path
    os.environ['SMILES']=str(smiles)
    !echo ${SMILES} > input.smi
    !obabel -ismi input.smi -O input.mol2 --gen3D --conformer --nconf 5000 --weighted 
    !babel -imol2 input.mol2 -oxyz input.xyz
    from ase.io import read, write
    inp1 = read('input.xyz')
    !acpype -i input.mol2 -c bcc -n 0 -m 1 -a gaff2 -f -o gmx -k "qm_theory='AM1',grms_tol=0.05,scfconv=1.d-10,ndiis_attempts=700, "

    import shutil
    src = './input.acpype/input_GMX.gro'
    copy = './input1.gro'
    shutil.copyfile(src,copy)
    src = './input.acpype/input_GMX.itp'
    copy = './input1.itp'
    shutil.copyfile(src,copy)

In [8]:
gaff_bcc(poly["Smiles"][0])

1 molecule converted
1 molecule converted
18 info messages 9 audit log messages 
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2022.1.3 (c) 2023 AWSdS |
==> ... charge set to 0
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
==> * Tleap OK *
==> Removing temporary files...
==> Using OpenBabel v.2.4.1

==> Writing GROMACS files

==> Disambiguating lower and uppercase atomtypes in GMX top file, even if identical.

==> Writing GMX dihedrals for GMX 4.5 and higher.

==> Writing pickle file input.pkl
==> Removing temporary files...
Total time of execution: 1s


In [10]:
def build_unitcell(gromacs_home,mol1name,density,max_atoms):

    # gromacs_home = gromacs_home
    # mol1name     = mol1name
    
    import pandas as pd
    import gc 
    
    #構造可視化
    import MDAnalysis as mda

    #混合溶液を作成
    import mdapackmol
    import numpy as np
    from ase import units

    # load individual molecule files
    mol1 = mda.Universe('input1.gro') # ここは多分読み込めてる？
    total_mol = int(max_atoms/(mol1.atoms.n_atoms))
    num_mols1 = total_mol
    mw_mol1 = np.sum(mol1.atoms.masses)
    total_weight = num_mols1 * mw_mol1 
    d = density / 1e24 # Density in g/Ang3 
    volume = (total_weight / units.mol) / d
    L = volume**(1.0/3.0)

    system = mdapackmol.packmol(
    [ mdapackmol.PackmolStructure(
    mol1, number=num_mols1,
    instructions=['inside box 0. 0. 0. '+str(L)+"  "+str(L)+"  "+str(L)]),
    ])
    
    system.atoms.write('mixture.gro')
    del system 
    gc.collect()
    
    import os 
    os.environ['GMX_MAXBACKUP'] = '-1'
    
    #gromacs_home = "/home/yamazaki/usr/local/gromacs/bin/"
    
    os.environ['OMP_NUM_THREADS'] = '1'
    
    commands = gromacs_home + "gmx editconf -f mixture.gro  -box "+ str(L/10.0)+"  "+str(L/10.0)+"  "+str(L/10.0) + "  " +" -o init.gro"

    import subprocess
    from subprocess import PIPE

    proc = subprocess.run(commands, shell=True, stdout=PIPE, stderr=PIPE,encoding='utf-8')
    output = proc.stdout
    print('STDOUT: editconf :: {}'.format(output))

    #make top file for GAFF

    top_file = "system.top"

    lines = [
        "; input_GMX.top created by acpype (v: 2020-07-25T09:06:13CEST) on Fri Jul 31 07:59:08 2020",
        ";by acpype (v: 2020-07-25T09:06:13CEST) on Fri Jul 31 07:59:08 2020",
        "   ",
        "[ defaults ]",
        "; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ",
        "1               2               yes             0.5     0.833",
        "    ",
        "; Include input.itp topology", 
        "#include \"input1.itp\"",
        "    ",
        "[ system ]",
        "input",
        "     ",
        "[ molecules ]",
        "; Compound        nmols" ,
        mol1name + "          " + str(int(num_mols1)), ]

    with open(top_file, mode='w') as f:
        f.write('\n'.join(lines))

In [6]:
#make mdp file for energy minimization
def make_mdp_em():

    mdp_file = "em.mdp"

    lines = [
    "; VARIOUS PREPROCESSING OPTIONS",
    ";title                    = Yo",
    ";cpp                      = /usr/bin/cpp",
    "include                  =", 
    "define                   =", 
    "    ",
    "; RUN CONTROL PARAMETERS",
    "integrator               = steep",
    "nsteps                   = 1000000",
    "emtol                    = 10",
    "emstep                   = 0.1",
    "nstlist                  = 1",
    "cutoff-scheme            = verlet",
    "vdw-type                 = cut-off",
    "rlist                    = 1.0",
    "rvdw                     = 1.0",
    "rcoulomb                 = 1.0",
    ]

    with open(mdp_file, mode='w') as f:
        f.write('\n'.join(lines))

#Equilibration of all the Replica 

#make mdp file for NVT run
def make_mdp_nvt(temp,steps,dt,cutoff,coulombtype):

    temperature      = temp
    simulation_steps = steps 
    time_step        = dt/1000.0  # ps
    cutoff_radius    = cutoff
    
    mdp_file = "run.mdp"

    lines = [
    "; VARIOUS PREPROCESSING OPTIONS",
    ";title                    = Yo",
    ";cpp                      = /usr/bin/cpp",
    "include                  =", 
    "define                   =", 
    "    ",
    "; RUN CONTROL PARAMETERS",
    "constraints               = none",
    ";constraints              = h-bonds",
    "niter =5000",
    "; Type of constraint algorithm",
    "constraint-algorithm     = Lincs",
    "integrator               = md",
    "nsteps                   = {}".format(simulation_steps),
    "dt                       = {}".format(time_step),
    "nstlist                  = 50",
    "rlist                    = {}".format(cutoff_radius),
    "rvdw                     = {}".format(cutoff_radius),
    "rcoulomb                 = {}".format(cutoff_radius),
    "coulombtype              = {}".format(coulombtype),
    "cutoff-scheme            = verlet",
    "vdw-type                 = cut-off",        
    "tcoupl                   = v-rescale",
    "tc-grps                  = system",
    "tau-t                    = 0.1",
    "ref-t                    = {}".format(temperature),
    "Pcoupl                   = no",
    "nstenergy                = 1000",
    "nstxout                  = 1000",
    "emtol                    = 10"
    ]

    with open(mdp_file, mode='w') as f:
        f.write('\n'.join(lines))

#make mdp file for production run
def make_mdp_run(temp,press,steps,dt,cutoff,coulombtype):

    temperature      = temp
    pressure         = press
    simulation_steps = steps 
    time_step        = dt/1000.0  # ps
    cutoff_radius    = cutoff
    
    mdp_file = "run.mdp"

    lines = [
    "; VARIOUS PREPROCESSING OPTIONS",
    ";title                    = Yo",
    ";cpp                      = /usr/bin/cpp",
    "include                  =", 
    "define                   =", 
    "    ",
    "; RUN CONTROL PARAMETERS",
    "constraints               = none",
    ";constraints              = h-bonds",
    "niter =5000",
    "; Type of constraint algorithm",
    "constraint-algorithm     = Lincs",
    "integrator               = md",
    "nsteps                   = {}".format(simulation_steps),
    "dt                       = {}".format(time_step),
    "nstlist                  = 50",
    "rlist                    = {}".format(cutoff_radius),
    "rvdw                     = {}".format(cutoff_radius),
    "rcoulomb                 = {}".format(cutoff_radius),
    "coulombtype              = {}".format(coulombtype),
    "cutoff-scheme            = verlet",
    "vdw-type                 = cut-off",        
    "tcoupl                   = v-rescale",
    "tc-grps                  = system",
    "tau-t                    = 0.1",
    "ref-t                    = {}".format(temperature),
    "Pcoupl                   = Berendsen",
    "tau-p                    = 2.5",
    "ref-p                    = {}".format(pressure),
    "compressibility          = 4.5e-5",
    "nstenergy                = 1000",
    "nstxout                  = 1000",
    "emtol                    = 10",
    "DispCorr                 = EnerPres",
    ]

    with open(mdp_file, mode='w') as f:
        f.write('\n'.join(lines))

In [20]:
def relax(gromacs_home,temps,inp_file,out_file,ncores,calc_nb,GPU_ID):
    
    gromacs_home=gromacs_home
    temps   = temps
    ncores  = ncores 
    calc_nb = calc_nb 
    GPU_ID  = GPU_ID
    
    import shutil
    import os 
    import subprocess
    from subprocess import PIPE
    import numpy as np
    
    os.environ['GMX_MAXBACKUP'] = '-1'
    #!. /home/yamazaki/usr/local/gromacs/bin/GMXRC
    
    #gromacs_home = "/home/yamazaki/usr/local/gromacs/bin/"
    
    #cut_off =0.3/0.4 (unstable) -> 0.6/0.8 20210726    
    conditions = [
        [100,   -1,   100000, "cut-off",1.0],  # step-0 NVT100K-100ps relaxization 
        [600,   -1,   500000, "cut-off",1.0],  # step-1 NVT1000K-500ps (50ps -> 500ps for chain stretching )
        [300,   -1,    50000, "cut-off",1.0],  # step-2 NVT300.0K-50ps
        [300, 1000,    50000, "cut-off",1.0],  # step-3 NPT1000atm-300.0K-50ps
        [600,   -1,    50000, "cut-off",1.0],  # step-4 NVT600K-50ps
        [300,   -1,   100000, "cut-off",1.0],  # step-5 NVT300.0K-100ps
        [300,10000,    50000, "cut-off",1.0],  # step-6 NPT30000atm-300.0K-50ps
        [600,   -1,    50000, "cut-off",1.0],  # step-7 NVT600K-50ps
        [300,   -1,   100000, "cut-off",1.0],  # step-8 NVT300.0K-100ps
        [300, 5000,    50000, "cut-off",1.0],  # step-9 NPT5000atm-300.0K-50ps
        [600,   -1,    50000, "cut-off",1.0],  # step-10 NVT600K-50ps
        [300,   -1,   100000, "pme",1.2],  # step-11 NVT300.0K-100ps
        [temps[0], 1,1100000, "pme",1.2],  # step-12 NPT1atm-300.0K-1000ps
    ]
    
    #energy minimization 
    
    make_mdp_em()
    
    #grompp (em.tpr作成, input1.itpとinput1.groが必要)
    os.environ['OMP_NUM_THREADS'] = '1'    
    commands = gromacs_home+"gmx grompp -f em.mdp -p system.top -c {}.gro -o em.tpr -maxwarn 10 ".format(inp_file)
    proc = subprocess.run(commands, shell=True, stdout=PIPE, stderr=PIPE,encoding='utf-8')
    output = proc.stdout
    print('STDOUT: grompp 1st :: {}'.format(output))
    #mdrun
    os.environ['OMP_NUM_THREADS'] = '{}'.format(ncores) 
    # commands = gromacs_home+"gmx mdrun -ntmpi 1 -s em.tpr -o em.trr -e em.edr -c em.gro -nb {0} -gpu_id {1}".format(calc_nb,GPU_ID)
    commands = gromacs_home+"gmx mdrun -ntmpi 1 -s em.tpr -o em.trr -e em.edr -c em.gro"
    proc = subprocess.run(commands, shell=True, stdout=PIPE, stderr=PIPE,encoding='utf-8')
    output = proc.stdout
    print('STDOUT: mdrun 1st :: {}'.format(output))

    src = './em.gro'
    copy = './input.gro'
    shutil.copyfile(src,copy)

    #relaxization 
    dt = 1.0
    #cutoff = 1.4 
    
    for count,cond in enumerate(conditions) :
        temperature = cond[0]
        pressure    = cond[1]
        md_steps    = cond[2]
        coul_type   = cond[3]
        cutoff      = cond[4]
        print("cond=",cond)
        if pressure < 0 :
            make_mdp_nvt(temperature,md_steps,dt,cutoff,coul_type)
        else :
            make_mdp_run(temperature,pressure,md_steps,dt,cutoff,coul_type)
            
        #grompp
        os.environ['OMP_NUM_THREADS'] = '1'    
        commands = gromacs_home+"gmx grompp -f run.mdp -p system.top -c input.gro -o run.tpr -maxwarn 10 "
        proc = subprocess.run(commands, shell=True, stdout=PIPE, stderr=PIPE,encoding='utf-8')
        output = proc.stdout
        print('STDOUT: grompp :: {}'.format(output))

        #mdrun
        os.environ['OMP_NUM_THREADS'] = '{}'.format(ncores)  
        if coul_type=="pme":
            # commands = gromacs_home+"gmx mdrun -ntmpi 1 -s run.tpr -o run.trr -e run.edr -c run.gro -bonded {0} -pme {0} -nb {0} -gpu_id {1} ".format(calc_nb,GPU_ID)
            commands = gromacs_home+"gmx mdrun -ntmpi 1 -s run.tpr -o run.trr -e run.edr -c run.gro" 
        else :
            # commands = gromacs_home+"gmx mdrun -ntmpi 1 -s run.tpr -o run.trr -e run.edr -c run.gro -bonded {0} -nb {0} -gpu_id {1} ".format(calc_nb,GPU_ID)
            commands = gromacs_home+"gmx mdrun -ntmpi 1 -s run.tpr -o run.trr -e run.edr -c run.gro"
        
        proc = subprocess.run(commands, shell=True, stdout=PIPE, stderr=PIPE,encoding='utf-8')
        output = proc.stdout
        print('STDOUT: mdrun :: {}'.format(output))
        
        src = './run.gro'
        copy = './input.gro'
        shutil.copyfile(src,copy)
        
        src = './run.gro'
        copy = './relax_{}.gro'.format(count)
        shutil.copyfile(src,copy)       
        
    
    src = './input.gro'
    copy = './{}.gro'.format(out_file)
    shutil.copyfile(src,copy)
    
    #PDB file の書き出し
    commands = "echo System > input.txt"
    proc = subprocess.run(commands, shell=True, stdout=PIPE, stderr=PIPE,encoding='utf-8')
    output = proc.stdout
    
    os.environ['OMP_NUM_THREADS'] = '1'    
    commands = gromacs_home+"gmx trjconv -s run.tpr  -f {0}.gro -pbc mol -conect -o {0}.pdb < input.txt".format(out_file)
    proc = subprocess.run(commands, shell=True, stdout=PIPE, stderr=PIPE,encoding='utf-8')
    output = proc.stdout
    print('STDOUT: trjconv :: {}'.format(output))
    
    line = np.array([float(p) for p in tail("{}.gro".format(out_file),1)[0]]) 
    v = line[0]*line[1]*line[2]
    
    return v

In [21]:
def tail(fn, n):
    # ファイルを開いてすべての行をリストで取得する
    with open(fn, 'r') as f:
        f.readline()
        lines = f.readlines()

    # 文字列を配列にしてから返す. ついでにstr->floatに型変換する
    return [line.strip().split() for line in lines[-n:]]

In [22]:
mol_name1 = "input"
build_unitcell(gromacs_home,mol_name1,density,max_atoms)
#build_unitcell(density,max_atoms)
inp_file  = "init"
out_file = "relaxed" 
temps = [300,]
ncores = 8
# calc_nb = "gpu"
calc_nb = "cpu"
GPU_ID = 0
volume = relax(gromacs_home,temps,inp_file,out_file,ncores,calc_nb,GPU_ID)
#volume = relax(input_file,output_file) 
print("relaxed system volume =",volume)

/Users/amano/anaconda3/envs/dieltools/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/amano/anaconda3/envs/dieltools/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/amano/anaconda3/envs/dieltools/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/amano/anaconda3/envs/dieltools/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  warnings.warn("Found no information for attr: '{}'"
/Users/amano/anaconda3/envs/dieltools/lib/p

STDOUT: editconf :: Note that major changes are planned in future for editconf, to improve usability and utility.
Read 37800 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found
    system size : 32.972 32.863 32.918 (nm)
    center      : 16.284 16.938 16.135 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  0.304 -0.350  0.453 (nm)
new center      : 16.588 16.588 16.588 (nm)
new box vectors : 33.176 33.176 33.176 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :36514.76               (nm^3)

STDOUT: grompp 1st :: Setting the LD random seed to -813877345

Generated 21 of the 21 non-bonded parameter combinations

Generated 21 of the 21 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'input'
Analysing residue names:
There are:   600      Other residues
Analysing residues not classified as Protein/DNA/RNA/Wa

In [23]:
#構造可視化(nglview版)
import nglview as nv
import ase.io
mol1 = ase.io.read('./relaxed.gro')
w = nv.show_ase(mol1)
w.add_label(radius=1,color="black",label_type="atom")
w.add_unitcell()
w.update_unitcell()
w

NGLWidget()

In [14]:
#密度
106.17*600/(121.60446826014802*6.02)/100

0.8701758846651405

In [24]:
import os 
import subprocess
from subprocess import PIPE
import numpy as np
!echo "Density" > input.txt
os.environ['OMP_NUM_THREADS'] = '1'    
commands = gromacs_home+"gmx energy -s run.tpr  -f run.edr  -b 100 -o density.xvg < input.txt"
proc = subprocess.run(commands, shell=True, stdout=PIPE, stderr=PIPE,encoding='utf-8')
proc.stdout

'\nStatistics over 1000001 steps [ 100.0000 through 1100.0000 ps ], 1 data sets\nAll statistics are over 10001 points\n\nEnergy                      Average   Err.Est.       RMSD  Tot-Drift\n-------------------------------------------------------------------------------\nDensity                      1005.7       0.72    2.35361    3.79939  (kg/m^3)\n'